In [10]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import plotly.express as px

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
# dataset file path

malignant_files = "dataset/trial/Malignant"
normal_files = "dataset/trial/Normal"

In [3]:
# function to load the images from the directories

def load_images(directory, label):
  images = []
  labels = []
  for filename in os.listdir(directory):
    if filename.endswith(".jpg") or filename.endswith(".png"):
      img_path = os.path.join(directory, filename)
      img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
      if img is not None:
        img = cv2.resize(img, (128, 128))
        images.append(img)
        labels.append(label)
  return images, labels

In [4]:
# normal = 0
# malignant = 1

malignant_images, malignant_labels = load_images(malignant_files, label=1)
normal_images, normal_labels = load_images(normal_files, label=0)

In [5]:
images = malignant_images + normal_images
labels = malignant_labels + normal_labels

In [6]:
images = np.array(images)
labels = np.array(labels)


print(len(images))

971


In [7]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [8]:
categories = {
    "Malignant": malignant_files,
    "Normal": normal_files
}

In [16]:
model = Sequential([
    Input(shape=(128, 128, 1)),

    # First Convolutional Block
    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    # Second Convolutional Block
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    # Third Convolutional Block
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    # Flatten and Fully Connected Layers
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),

    # Output Layer
    Dense(2, activation='softmax')
])

# Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

y_train_encoded = to_categorical(y_train, num_classes = 2)
y_test_encoded = to_categorical(y_test, num_classes = 2)

# Train the Model
history = model.fit(x_train, y_train_encoded, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None, 2, 2), output.shape=(None, 2)

In [15]:
y_test_encoded = to_categorical(y_test, num_classes=2)

test_loss, test_acc = model.evaluate(x_test, y_test_encoded)
#print(f'Test Loss: {test_loss}')
#print(f'Test Accuracy: {test_acc}')

print("Model Accuracy =",test_acc*100)

ValueError: Cannot take the length of shape with unknown rank.